In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
root_dir = r'C:\rkka_Projects\cell_death_v1\Data/pathway/collected/augmented'
test_dir = r'C:\rkka_Projects\cell_death_v1\Data/pathway/collected/test/augmented'

In [4]:
# Define a custom dataset class
class AugmentedCellDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Collect images from 'live' and 'dead' folders
        for label, class_name in enumerate(['Apoptosis', 'Necroptosis', 'Necrosis']):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.image_paths.append(img_path)
                self.labels.append(label)  # 0 for live, 1 for dead

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [5]:
# Define a custom dataset class
class UnseenAugmentedCellDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        # Collect images from 'live' and 'dead' folders
        for label, class_name in enumerate(['Apoptosis', 'Necroptosis', 'Necrosis']):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.image_paths.append(img_path)
                self.labels.append(label)  # 0 for live, 1 for dead

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [13]:
# Create dataset
train_dataset = AugmentedCellDataset(root_dir=root_dir, transform=models.EfficientNet_V2_L_Weights.IMAGENET1K_V1.transforms())
validation_dataset = UnseenAugmentedCellDataset(root_dir=test_dir, transform=models.EfficientNet_V2_L_Weights.IMAGENET1K_V1.transforms())
# Split dataset into training and validation
# test_size = 0.3
# batch_size = 16
# train_idx, val_idx = train_test_split(range(len(dataset)), test_size=test_size, stratify=dataset.labels)
# train_set = torch.utils.data.Subset(dataset, train_idx)
# val_set = torch.utils.data.Subset(dataset, val_idx)

# Create dataloaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

<h1> Only last layer </h1>

In [20]:
from torchvision import models
# Load pretrained model
model = models.efficientnet_v2_l(weights=models.EfficientNet_V2_L_Weights)

# Freeze parameters except last layer(fc layer)
for param in model.parameters():
    param.requires_grad = False
    
# Modify the final layer for binary classification (2 classes: live, dead)
model.classifier[1] = nn.Linear(in_features=model.classifier[1].in_features, out_features=3)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [22]:
# Training loop
num_epochs = 30
for epoch in tqdm(range(num_epochs)):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Calculate statistics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    # Print training stats
    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")

    # Validation
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Calculate accuracy
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss /= len(val_loader)
    val_acc = 100 * correct / total
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%\n")
    
    # Unseen
    model.eval()  # Set model to evaluation mode
    unseen_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in unseen_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            unseen_loss += loss.item()

            # Calculate accuracy
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    unseen_loss /= len(unseen_loader)
    unseen_acc = 100 * correct / total
    print(f"Unseen Test Loss: {unseen_loss:.4f}, Unseen Test Accuracy: {unseen_acc:.2f}%\n")
    
    os.makedirs('finetuning/pathway', exist_ok=True)
    # torch.save(model.state_dict(), 'finetuning/pathway/effv2_finetuned_epoch_'+str(epoch).zfill(2)+'_accuracy_'+str(val_acc)[:5]+'.pth')

print("Training complete!")

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch [1/30], Loss: 0.1425, Accuracy: 95.66%
Validation Loss: 0.0675, Validation Accuracy: 98.82%



  3%|▎         | 1/30 [00:19<09:14, 19.12s/it]

Unseen Test Loss: 0.8463, Unseen Test Accuracy: 75.76%

Epoch [2/30], Loss: 0.1992, Accuracy: 94.39%
Validation Loss: 0.0712, Validation Accuracy: 99.41%



  7%|▋         | 2/30 [00:37<08:47, 18.82s/it]

...

Unseen Test Loss: 1.0343, Unseen Test Accuracy: 75.76%

Epoch [6/30], Loss: 0.1132, Accuracy: 96.94%
Validation Loss: 0.0296, Validation Accuracy: 100.00%



 20%|██        | 6/30 [01:53<07:34, 18.92s/it]

Unseen Test Loss: 0.9842, Unseen Test Accuracy: 75.76%



 20%|██        | 6/30 [01:56<07:44, 19.35s/it]


KeyboardInterrupt: 

In [18]:
# Unseen
model.eval()  # Set model to evaluation mode
unseen_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels in unseen_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        unseen_loss += loss.item()

        # Calculate accuracy
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

unseen_loss /= len(unseen_loader)
unseen_acc = 100 * correct / total
print(f"Unseen Test Loss: {unseen_loss:.4f}, Unseen Test Accuracy: {unseen_acc:.2f}%\n")

Unseen Test Loss: 0.9063, Unseen Test Accuracy: 76.97%

